In [ ]:
import netrc

OUTPUT="/Users/mmodat/Data/temp_ifind"
SERVER="https://int-xnat01.isd.kcl.ac.uk"
PROJECT="FHEART"

USER, _, PWD = netrc.netrc().authenticators(SERVER)

In [ ]:
import pandas as pd
import requests
import json
import os

# Download the list of existing sessions on XNAT
reqSession = requests.session()
url = "{}/data/projects/{}/experiments?format=json".format(SERVER, PROJECT)
r = reqSession.get(url,
                  verify=False,
                  auth=(USER,
                        PWD))
reqSession.close()
raw_data = pd.DataFrame(r.json()['ResultSet']['Result'])

In [ ]:
reqSession = requests.session()

# Loop over all the session already on XNAT
for i, row in raw_data[['ID', 'label']].iterrows():
    id = row[0]
    session = row[1]
    subject, session = session.split('_')
    subject_folder = os.path.join(OUTPUT, subject)
    session_folder = os.path.join(subject_folder, session)
    # Create folders for subject and session if they don't already exist
    if not os.path.isdir(subject_folder):
        os.mkdir(subject_folder)
    if not os.path.isdir(session_folder):
        os.mkdir(session_folder)
        # Pull the list of files for current session
        url = "{}/data/projects/{}/subjects/{}/experiments/{}/scans/ALL/files".format(
            SERVER,
            PROJECT,
            subject,
            id)
        r = reqSession.get(url,
                           verify=False,
                           auth=(USER,
                                 PWD))

        file_data = pd.DataFrame(r.json()['ResultSet']['Result'])[['URI', 'Name']]

        # Download the actual files one at the time to avoid issues with too large files
        for j, row_file in file_data.iterrows():
            url = "{}{}".format(SERVER, row_file[0])
            r = reqSession.get(url,
                               verify=False,
                               auth=(USER,
                                     PWD))
            with open(os.path.join(session_folder, row_file[1]), 'wb') as f:
                f.write(r.content)
            break
        
reqSession.close()